In [1]:
import pandas as pd

# Reading movies lens dataset
- column names and seprater as per readme file

In [2]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ratings.dat', sep='::', names=r_cols,
                      encoding='latin-1')

/home/aakash/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


# Creating a sequence of movie watched
- Since data is not sorted by timestamp
- first sorting the data based on timestamp.

In [3]:
rating_data = ratings.set_index(['user_id','unix_timestamp']).sort_index()

# Loading meta data

In [4]:
r_cols = ['movie_id', 'movie_name', 'movie_genre']

metadata = pd.read_csv('movies.dat', sep='::', names=r_cols,
                      encoding='latin-1', index_col='movie_id')
metadata.head()

/home/aakash/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


,movie_name,movie_genre
movie_id,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy


# Taking only those movies which user likes

In [5]:
rating_data=rating_data[rating_data.rating>3]

In [6]:
movie_sequence = rating_data.groupby('user_id')['movie_id'].apply(list)

# Filtering out users who have less 10 liked movies

In [7]:
movie_sequence = movie_sequence.reset_index()
movie_sequence['Length'] = movie_sequence['movie_id'].str.len()
movie_sequence = movie_sequence[movie_sequence.Length > 15]
movie_sequence.head()

,user_id,movie_id,Length
0,1,"[3186, 1270, 1721, 1022, 1836, 3408, 2804, 120...",45
1,2,"[1198, 1210, 1293, 2943, 1225, 1193, 318, 3030...",73
2,3,"[2858, 1968, 1961, 1266, 1378, 1379, 3671, 590...",37
3,4,"[1097, 3468, 480, 260, 1198, 1387, 2028, 2366,...",18
4,5,"[908, 919, 1250, 2858, 2997, 2770, 2355, 2908,...",82


In [8]:
movie_sequence.describe()

,user_id,Length
count,5580.000000,5580.000000
mean,3026.947849,102.119713
std,1745.113817,106.363510
min,1.000000,16.000000
25%,1509.750000,33.000000
50%,3036.500000,65.000000
75%,4542.250000,130.000000
max,6040.000000,1435.000000


# Creating Input for PageRank.

In [9]:
def input_output_splitter(data):
    return pd.Series([data[0], data[1:]])

In [10]:
movie_sequence[['input', 'output']] = movie_sequence['movie_id'].apply(input_output_splitter)

In [11]:
movie_sequence.head()

,user_id,movie_id,Length,input,output
0,1,"[3186, 1270, 1721, 1022, 1836, 3408, 2804, 120...",45,3186,"[1270, 1721, 1022, 1836, 3408, 2804, 1207, 119..."
1,2,"[1198, 1210, 1293, 2943, 1225, 1193, 318, 3030...",73,1198,"[1210, 1293, 2943, 1225, 1193, 318, 3030, 2858..."
2,3,"[2858, 1968, 1961, 1266, 1378, 1379, 3671, 590...",37,2858,"[1968, 1961, 1266, 1378, 1379, 3671, 590, 260,..."
3,4,"[1097, 3468, 480, 260, 1198, 1387, 2028, 2366,...",18,1097,"[3468, 480, 260, 1198, 1387, 2028, 2366, 1201,..."
4,5,"[908, 919, 1250, 2858, 2997, 2770, 2355, 2908,...",82,908,"[919, 1250, 2858, 2997, 2770, 2355, 2908, 3016..."


# Initalizing all ranks to 1

In [12]:
movies_df = movie_sequence[['input', 'output']]

# Initalizing all rank to 1
movies_df['rank'] = 1

/home/aakash/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
movies_df.head()

,input,output,rank
0,3186,"[1270, 1721, 1022, 1836, 3408, 2804, 1207, 119...",1
1,1198,"[1210, 1293, 2943, 1225, 1193, 318, 3030, 2858...",1
2,2858,"[1968, 1961, 1266, 1378, 1379, 3671, 590, 260,...",1
3,1097,"[3468, 480, 260, 1198, 1387, 2028, 2366, 1201,...",1
4,908,"[919, 1250, 2858, 2997, 2770, 2355, 2908, 3016...",1


# Code for dividing rank

In [14]:
def distribute_rank(list_of_cids):
    total_length = len(list_of_cids)
    for item in list_of_cids:
        rank_list.append((item,(round(rank_dict.get(item,1)/total_length,2))))

# Iterate and Generate final rank

In [ ]:
rank_dict = {}
rank_list = []
DAMPING_FACTOR = 0.85
CONSTANT_TERM = 0.15

# Iterating over graph 10 time
for i in range(5):
    for item in movies_df['output'][100:300]:
        distribute_rank(item)
        for tuple_cid_rank in rank_list:
            rank_dict.update({tuple_cid_rank[0]:round(tuple_cid_rank[1]+rank_dict.get(tuple_cid_rank[0],1)*DAMPING_FACTOR+CONSTANT_TERM,2)})
        
        

# Sorting based on score

In [ ]:
movies_sorted_list = [(k,v) for k, v in sorted(rank_dict.items(), key=lambda item: item[1], reverse=True)]
top_10_keys = []
for key, val in movies_sorted_list[:10]:
    print(key,val)
    top_10_keys.append(key)
    

In [ ]:
top_10_keys = []
for key, val in movies_sorted_list[:10]:
    print(key,val)
    top_10_keys.append(key)
    

In [ ]:
metadata.loc[top_10_keys]